In [98]:
# %pip install -U llama-index
# %pip install -U llama-index-core
# %pip install llama-index-embeddings-openai
# %pip install llama-index-postprocessor-flag-embedding-reranker
# %pip install git+https://github.com/FlagOpen/FlagEmbedding.git
# %pip install -U llama-parse
# %pip install -U llama-index-llms-llama-cpp
# %pip install -U llama-index-core-llms
# %pip install -U llama-index-embeddings-openai

In [99]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-SyHQUKzf4guxK5aU1QnzURVVl4sbQ4pqX5QzFh7SXhytaj8N"

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "sk-proj-Q0eWC0bJfIIaLkIGOaHOT3BlbkFJcAPgSc2VZCGKEPVJVtzI"

In [100]:
from llama_parse import LlamaParse

parser_gpt4o = LlamaParse(
    result_type="markdown",
    # api_key=api_key,
    gpt4o_mode=True,
    # gpt4o_api_key="<gpt4o_api_key>"
)

In [101]:
import os
import glob

# Define the folder containing the PDF files
data_folder = './data/'

# Use glob to find all PDF files in the folder
pdf_files = glob.glob(os.path.join(data_folder, '*.pdf'))
pdf_files

['./data\\Market Report_02 May 2024.pdf',
 './data\\Market Report_03 May 2024.pdf',
 './data\\Market Report_06 May 2024.pdf',
 './data\\Market Report_07 May 2024.pdf',
 './data\\Market Report_08 May 2024.pdf',
 './data\\Market Report_09 May 2024.pdf',
 './data\\Market Report_10 May 2024.pdf',
 './data\\Market Report_13 May 2024.pdf',
 './data\\Market Report_14 May 2024.pdf',
 './data\\Market Report_15 May 2024.pdf',
 './data\\Market Report_16 April 2024.pdf',
 './data\\Market Report_16 May 2024.pdf',
 './data\\Market Report_17 May 2024.pdf',
 './data\\Market Report_20 May 2024.pdf',
 './data\\Market Report_21 May 2024.pdf']

In [102]:
documents_gpt4o = parser_gpt4o.load_data(pdf_files)

Parsing files:   0%|          | 0/15 [00:00<?, ?it/s]

Error while parsing the file './data\Market Report_17 May 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}


Parsing files:   7%|▋         | 1/15 [00:03<00:54,  3.92s/it]

Error while parsing the file './data\Market Report_10 May 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}
Error while parsing the file './data\Market Report_13 May 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}


Parsing files:  13%|█▎        | 2/15 [00:04<00:28,  2.19s/it]

...

Error while parsing the file './data\Market Report_20 May 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}


Parsing files:  87%|████████▋ | 13/15 [00:17<00:01,  1.14it/s]

Error while parsing the file './data\Market Report_08 May 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}


Parsing files: 100%|██████████| 15/15 [00:18<00:00,  1.22s/it]

Error while parsing the file './data\Market Report_15 May 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}
Error while parsing the file './data\Market Report_16 April 2024.pdf': Failed to parse the file: {"detail":"You've exceeded the maximum number of pages you can parse in a day (1000). Please contact support to increase your limit."}


In [103]:
print(documents_gpt4o[0].get_content())

IndexError: list index out of range

In [104]:
from copy import deepcopy
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex


def get_nodes(docs):
    """Split docs into nodes, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            nodes.append(node)

    return nodes

In [105]:
# this will split into pages
nodes = get_nodes(documents_gpt4o)

In [106]:
vector_index = VectorStoreIndex(nodes)

In [107]:
query_engine = vector_index.as_query_engine(similarity_top_k=10)

In [108]:
response = query_engine.query(
    "Corporate Bonds Board, a 5-year bond?"
)

In [109]:
print(str(response))

Empty Response


In [110]:
response = query_engine.query("What are the prices offered for CRDB in all days in May 2024? Output should be in the json as follow {{'date': date , 'price':price}, {'date': date , 'price':price}}?")
print(str(response))

Empty Response


In [ ]:
import json
import re
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Use regular expressions to find all JSON objects in the string
json_objects = re.findall(r'\{[^}]+\}', str(response))

# Parse each JSON object and convert it to a dictionary
data_list = [json.loads(json_obj) for json_obj in json_objects]

In [ ]:
# Convert date strings to datetime objects and sort the data
sorted_data = sorted(data_list, key=lambda x: datetime.strptime(x['date'], '%d %b %Y'))

# Extract sorted dates and prices
dates = [datetime.strptime(item['date'], '%d %b %Y') for item in sorted_data]
prices = [item['price'] for item in sorted_data]

# Plot the data
plt.figure(figsize=(10, 5))
# plt.plot(dates, prices, marker='o', linestyle='-', color='b')
plt.scatter(dates, prices, marker='o', linestyle='-', color='b')

# Formatting the plot
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Date vs Price')
plt.grid(True)
plt.gcf().autofmt_xdate()  # Auto-format date labels
plt.show()